In [1]:
import os

import weave

os.environ["WF_TRACE_SERVER_URL"] = "http://127.0.01:6345"

client = weave.init("action_test_6")

Logged in as Weights & Biases user: timssweeney.
View Weave data at https://wandb.ai/timssweeney/action_test_6/weave


In [3]:
from openai import OpenAI


@weave.op
def extract_name(user_input: str) -> str:
    openai_client = OpenAI()
    if "Tim" in user_input:
        return "i don't know!"
    response = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "Extract the name from the user input. If there is no name, return an empty string.",
            },
            {"role": "user", "content": user_input},
        ],
        temperature=0.0,
        max_tokens=64,
        top_p=1,
    )
    return response.choices[0].message.content


calls = []
names = ["Scott", "Adrian", "Tim", "Jeff", "Shawn"]
for name in names:
    res, call = extract_name.call(f"My name is {name}.")
    calls.append(call)

print(calls)

🍩 https://wandb.ai/timssweeney/action_test_6/r/call/0192bd55-2347-7bd3-9a5b-b67569c5cc3f
🍩 https://wandb.ai/timssweeney/action_test_6/r/call/0192bd55-24f4-7500-82df-4d40849552c3
🍩 https://wandb.ai/timssweeney/action_test_6/r/call/0192bd55-268f-7392-a630-f1439b06dc63
🍩 https://wandb.ai/timssweeney/action_test_6/r/call/0192bd55-2695-7fd2-87e6-cda7c6a20104
🍩 https://wandb.ai/timssweeney/action_test_6/r/call/0192bd55-282b-7eb1-8c3c-72c41532f698
[Call(_op_name=<Future at 0x2ad619a20 state=finished returned str>, trace_id='0192bd55-2347-7bd3-9a5b-b68d23f243fd', project_id='timssweeney/action_test_6', parent_id=None, inputs={'user_input': 'My name is Scott.'}, id='0192bd55-2347-7bd3-9a5b-b67569c5cc3f', output='Scott', exception=None, summary={'usage': {'gpt-3.5-turbo-0125': {'requests': 1, 'completion_tokens': 1, 'prompt_tokens': 35, 'total_tokens': 36, 'completion_tokens_details': {'reasoning_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0}}}}, display_name=None, attributes=Attribu